In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#Load MoveNet model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/thunder/4')
movenet = model.signatures['serving_default']

In [ ]:
# Improved keypoint processing with augmentation
def process_image(image):
    image = tf.image.resize_with_pad(image, 256, 256)  # Higher resolution for Thunder
    return tf.cast(image, dtype=tf.int32)

def extract_keypoints(image):
    image = tf.convert_to_tensor(image)
    image = process_image(image)
    outputs = movenet(tf.expand_dims(image, axis=0))
    keypoints = outputs['output_0'].numpy()[0][0]
    
    # Confidence check and filtering
    if np.mean(keypoints[:,2]) < 0.2:  # Reject low-confidence detections
        return None
    return keypoints



In [ ]:
# Robust normalization with augmentation
def normalize_and_augment(keypoints):
    # Center around mid-hip
    hip = (keypoints[11] + keypoints[12]) / 2  # Average left/right hips
    normalized = keypoints - hip
    
    # Scale based on shoulder distance
    shoulders = normalized[5] - normalized[6]
    scale = np.linalg.norm(shoulders) + 1e-7
    normalized /= scale
    
    # Data augmentation
    if np.random.rand() > 0.5:  # 50% chance to augment
        # Add noise
        normalized += np.random.normal(0, 0.05, normalized.shape)
        # Flip horizontally
        normalized[:,0] *= -1
        
    return normalized.flatten()

In [ ]:
# Dataset preparation with auto-augmentation
def prepare_dataset(data_dir, augment_factor=5):
    X = []
    y = []
    class_names = sorted(os.listdir(data_dir))
    
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            keypoints = extract_keypoints(img)
            if keypoints is None:
                continue
                
            # Original sample
            normalized = normalize_and_augment(keypoints)
            X.append(normalized)
            y.append(class_idx)
            
            # Generate augmented samples
            for _ in range(augment_factor):
                augmented = normalize_and_augment(keypoints)
                X.append(augmented)
                y.append(class_idx)
    
    return np.array(X), np.array(y), class_names

In [ ]:
# Replace with your dataset path
X, y, class_names = prepare_dataset('C:/Users/5A_Traders/Downloads/FYP_ON_DEV/FYP_IntelliTrain/PoseEstimation/Dataset/archive')

In [ ]:
# Step 6: Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Optimized model for small datasets
classifier = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(51,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Custom learning schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
classifier.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Enhanced training with validation
history = classifier.fit(
    X_train, y_train,
    epochs=50,
    batch_size=8,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ]
)

In [ ]:
# Live prediction with confidence threshold
def live_prediction(class_names, confidence_threshold=0.6):
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Process frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        keypoints = extract_keypoints(rgb_frame)
        
        if keypoints is not None:
            features = normalize_and_augment(keypoints).reshape(1, -1)
            proba = classifier.predict(features, verbose=0)[0]
            
            if np.max(proba) > confidence_threshold:
                class_idx = np.argmax(proba)
                label = f"{class_names[class_idx]} ({np.max(proba):.1f})"
            else:
                label = "Unknown pose"
            
            # Draw keypoints
            for y, x, conf in keypoints:
                if conf > 0.3:
                    cv2.circle(frame, (int(x*frame.shape[1]), int(y*frame.shape[0])), 
                               5, (0,255,0), -1)
            
            cv2.putText(frame, label, (10,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        cv2.imshow('Pose Estimation', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

live_prediction(class_names)